In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tensorflow as tf

hello = tf.constant("hello tensorflow!")
print(hello)


tf.Tensor(b'hello tensorflow!', shape=(), dtype=string)


In [2]:
import tensorflow as tf

x = 2
y = 3
z = tf.add(x, y, name ='Add')

In [3]:
#direct dependency
a = tf.multiply(8,5)
b = tf.multiply(a,1)

In [4]:
#indirect dependency
a = tf.multiply(8,5)
b = tf.multiply(4,3)

In [8]:
#create a graph
graph = tf.Graph()

with graph.as_default():
    z = tf.add(x, y , name='Add')
    print(z)

Tensor("Add:0", shape=(), dtype=int32)


In [17]:
#clear default graph
# tf.compat.v1.reset_default_graph()

Sessions

In [9]:

sess = tf.compat.v1.Session()

In [2]:
import tensorflow as tf


a = tf.multiply(3, 3)

with tf.compat.v1.Session() as sess:
    result = sess.run(a)
    print(result)

9


Variables

In [5]:

x = tf.Variable(13)


In [ ]:
import tensorflow as tf


W = tf.Variable(tf.random.normal([500,111], stddev =0.35), name ="weights")
print(W.numpy())

tf.compat.v1.global_variables_initializer()


In [6]:
x = tf.Variable(1212)
init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as sess:
    sess.run(init)
    print(sess.run(x))

1212


Constants

In [12]:
import tensorflow as tf

x = tf.constant(13)

Placeholder

In [8]:
import tensorflow as tf

# Disable eager execution
tf.compat.v1.disable_eager_execution()

# Use placeholder as in TensorFlow 1.x
x = tf.compat.v1.placeholder("float", None)
y = x + 3

with tf.compat.v1.Session() as sess:
    result = sess.run(y, feed_dict={x: 2.0})
    print(result)


5.0


In [9]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
with tf.compat.v1.Session() as sess:
    result = sess.run(y, feed_dict={x: 5})
    print(result)


8.0


Introducing TensorBoard

In [11]:
import tensorflow as tf

x = tf.constant(1,name='x')
y = tf.constant(1,name='y')
a = tf.constant(3,name='a')
b = tf.constant(3,name='b')

prod1 = tf.multiply(x,y, name='prod1')
prod2 = tf.multiply(a,b,name='prod2')

sum = tf.add(prod1,prod2,name='sum')

with tf.compat.v1.Session() as sess:
  writer = tf.compat.v1.summary.FileWriter(logdir='./graphs',graph=sess.graph)
  print(sess.run(sum))

10


Creating a name scope

In [15]:
import tensorflow as tf

x = tf.constant(1,name='x')
y = tf.constant(1,name='y')
a = tf.constant(3,name='a')
b = tf.constant(3,name='b')

with tf.compat.v1.name_scope("Product"):
    with tf.compat.v1.name_scope("prod1"):
        prod1 = tf.multiply(x, y, name="prod1")
    with tf.compat.v1.name_scope("prod2"):
        prod2 = tf.multiply(a, b, name="prod2")

with tf.compat.v1.name_scope("Sum"):
    sum_op = tf.add(prod1,prod2,name='sum')

with tf.compat.v1.Session() as sess:
    writer = tf.compat.v1.summary.FileWriter('./graphs',sess.graph)
    print(sess.run(sum))

10


Handwritten digit classification using Tensorflow

In [18]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Disable eager execution for TensorFlow 1.x style
tf.compat.v1.disable_eager_execution()

# Load and preprocess the MNIST dataset
mnist_data, mnist_info = tfds.load('mnist', split='train', as_supervised=True, with_info=True)

# Normalize the images
def normalize_img(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Prepare the dataset for training: batch and shuffle
batch_size = 128
mnist_data = mnist_data.map(normalize_img)
mnist_data = mnist_data.cache() \
                       .shuffle(buffer_size=10000) \
                       .batch(batch_size) \
                       .repeat()  # Repeat the dataset for multiple epochs

# Create an iterator for the dataset in TensorFlow 1.x style
iterator = tf.compat.v1.data.make_one_shot_iterator(mnist_data)
next_batch = iterator.get_next()

# Define placeholders for input and output
X = tf.compat.v1.placeholder(tf.float32, [None, 28, 28, 1], name="input")  # Modified shape to include channel
Y = tf.compat.v1.placeholder(tf.int64, [None], name="label")

# Flatten the input using Keras Flatten layer
X_flattened = tf.keras.layers.Flatten()(X)

# Build the model using `tf.keras.layers.Dense`
with tf.compat.v1.name_scope("Model"):
    layer_1 = tf.keras.layers.Dense(512, activation='relu')(X_flattened)
    layer_2 = tf.keras.layers.Dense(256, activation='relu')(layer_1)
    layer_3 = tf.keras.layers.Dense(128, activation='relu')(layer_2)
    logits = tf.keras.layers.Dense(10)(layer_3)  # No activation for logits

# Define loss and optimizer
with tf.compat.v1.name_scope("Loss"):
    loss = tf.compat.v1.losses.sparse_softmax_cross_entropy(labels=Y, logits=logits)

with tf.compat.v1.name_scope("Optimizer"):
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)

# Define accuracy metric
with tf.compat.v1.name_scope("Accuracy"):
    correct_pred = tf.equal(tf.argmax(logits, 1), Y)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize all variables
init = tf.compat.v1.global_variables_initializer()

# Training parameters
num_epochs = 10
num_steps_per_epoch = mnist_info.splits['train'].num_examples // batch_size

with tf.compat.v1.Session() as sess:
    sess.run(init)

    for epoch in range(num_epochs):
        total_accuracy = 0
        total_loss = 0

        for step in range(num_steps_per_epoch):
            batch = sess.run(next_batch)
            batch_x, batch_y = batch[0], batch[1]
            
            # Reshape batch_x to have 4D shape (batch_size, 28, 28, 1)
            batch_x = batch_x.reshape(-1, 28, 28, 1)
            
            _, acc, l = sess.run([optimizer, accuracy, loss], feed_dict={X: batch_x, Y: batch_y})
            
            total_accuracy += acc
            total_loss += l

        print(f'Epoch {epoch+1}, Loss: {total_loss/num_steps_per_epoch:.4f}, Accuracy: {total_accuracy/num_steps_per_epoch:.4f}')
                             

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:01<00:00,  2.96 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:02<00:00,  2.50 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:02<00:00,  1.82 url/s]


Dataset mnist downloaded and prepared to C:\Users\shubh\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.



Epoch 1, Loss: 0.5619, Accuracy: 0.8596
Epoch 2, Loss: 0.1977, Accuracy: 0.9439
Epoch 3, Loss: 0.1458, Accuracy: 0.9580
Epoch 4, Loss: 0.1152, Accuracy: 0.9673
Epoch 5, Loss: 0.0934, Accuracy: 0.9730
Epoch 6, Loss: 0.0776, Accuracy: 0.9780
Epoch 7, Loss: 0.0653, Accuracy: 0.9811
Epoch 8, Loss: 0.0567, Accuracy: 0.9839
Epoch 9, Loss: 0.0471, Accuracy: 0.9871
Epoch 10, Loss: 0.0409, Accuracy: 0.9887


Math operations in tensorflow

In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
tf.compat.v1.enable_eager_execution()

In [3]:
x = tf.compat.v1.constant([1.,2.,3.])
y = tf.compat.v1.constant([3.,2.,1.])

Basic Operations

Addition

In [4]:
sum = tf.add(x,y)
sum.numpy()

array([4., 4., 4.], dtype=float32)

Subtraction

In [5]:
difference = tf.subtract(x,y)
difference.numpy()

array([-2.,  0.,  2.], dtype=float32)

Multiplication

In [6]:
product = tf.multiply(x,y)
product.numpy()

array([3., 4., 3.], dtype=float32)

Division

In [7]:
division = tf.divide(x,y)
division.numpy()

array([0.33333334, 1.        , 3.        ], dtype=float32)

Dot Product

In [8]:
dot_product = tf.reduce_sum(tf.multiply(x,y))

dot_product.numpy()

10.0

Finding the index of min and max element

In [9]:
x = tf.constant([10,0,13,9])

In [10]:
#index of min value:
tf.argmin(x).numpy()

1

In [11]:
#index of max value:
tf.argmax(x).numpy()

2

Squared Difference

In [12]:
x = tf.compat.v1.Variable([1,3,5,7,11])
y = tf.compat.v1.Variable([1])

tf.math.squared_difference(x,y).numpy()

array([  0,   4,  16,  36, 100])

Power

In [13]:
#x^x
x = tf.compat.v1.Variable([1,2,3,4])
tf.pow(x,x).numpy()

array([  1,   4,  27, 256])

Rank of the Matrix

In [14]:
x = tf.compat.v1.constant([[[1,2,4],[3,4,5]],
             [[1,2,4],[3,4,5]]])

x.shape

TensorShape([2, 2, 3])

In [15]:
x.numpy()

array([[[1, 2, 4],
        [3, 4, 5]],

       [[1, 2, 4],
        [3, 4, 5]]])

Reshape the Matrix

In [16]:
x = tf.compat.v1.constant([[1,2,3,4],[5,6,7,8]])

In [17]:
tf.reshape(x,[8,1]).numpy()

array([[1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8]])

Transpose the Matrix

In [18]:
tf.transpose(x)

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[1, 5],
       [2, 6],
       [3, 7],
       [4, 8]])>

Typecasting

In [19]:
x.dtype

tf.int32

In [21]:
x = tf.cast(x, dtype=tf.float32)

x.dtype

tf.float32

Concatenating two matrices

In [22]:
x = [[3,6,9], [7,7,7]]
y = [[4,5,6], [5,5,5]]

In [23]:
#concatenate row-wise
tf.concat([x,y],0).numpy()

array([[3, 6, 9],
       [7, 7, 7],
       [4, 5, 6],
       [5, 5, 5]])

In [24]:
#concatenate column-wise:
tf.concat([x,y],1).numpy()

array([[3, 6, 9, 4, 5, 6],
       [7, 7, 7, 5, 5, 5]])

In [25]:
#Stack x matrix:
tf.stack(x, axis=1).numpy()

array([[3, 7],
       [6, 7],
       [9, 7]])

Reduce Mean

In [26]:
x = tf.compat.v1.Variable([[1.0,5.0], [2.0,3.0]])

In [27]:
x.numpy()

array([[1., 5.],
       [2., 3.]], dtype=float32)

In [28]:
#compute average values 
tf.reduce_mean(input_tensor=x).numpy()

2.75

In [29]:
#average across the row
tf.reduce_mean(input_tensor=x, axis =0).numpy()

array([1.5, 4. ], dtype=float32)

In [30]:
#average across the column
tf.reduce_mean(input_tensor=x,axis=1,keepdims=True).numpy()

array([[3. ],
       [2.5]], dtype=float32)

Reduce Sum

In [31]:
x.numpy()

array([[1., 5.],
       [2., 3.]], dtype=float32)

In [32]:
#sum values across rows
tf.reduce_sum(x,0).numpy()

array([3., 8.], dtype=float32)

In [33]:
#sum values across columns
tf.reduce_sum(x,1).numpy()

array([6., 5.], dtype=float32)

In [35]:
#sum all the values:
tf.reduce_sum(x,[0,1]).numpy()

11.0

Drawing Random Values

In [36]:
#drawing values from normal the normal distribution:
tf.random.normal(shape=(3,2),mean=10.0,stddev=2.0).numpy()

array([[10.590439 , 11.161395 ],
       [13.4485035,  9.709789 ],
       [ 8.874537 , 11.036206 ]], dtype=float32)

In [37]:
#drawing values from uniform distribution
tf.random.uniform(shape=(3,2), minval=0, maxval=None,dtype=tf.float32,).numpy()

array([[0.8945638 , 0.7832072 ],
       [0.5405601 , 0.95168924],
       [0.8249128 , 0.19475007]], dtype=float32)

Create 0's and 1's

In [38]:
tf.zeros([5,5]).numpy()

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)

In [39]:
tf.zeros_like(x).numpy()

array([[0., 0.],
       [0., 0.]], dtype=float32)

In [41]:
tf.ones([5,5]).numpy()

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]], dtype=float32)

In [42]:
tf.ones_like(x).numpy()

array([[1., 1.],
       [1., 1.]], dtype=float32)

Compute Softmax Probabilities

In [43]:
x = tf.constant([7.,2.,5.])
tf.nn.softmax(x).numpy()

array([0.8756006 , 0.00589975, 0.11849965], dtype=float32)

Create Identity Matrix

In [44]:
i_matrix = tf.eye(7)
print(i_matrix.numpy())

[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]


L2 Normalization

In [45]:
tf.math.l2_normalize(x,axis=None, epsilon=1e-12,).numpy()

array([0.79259396, 0.22645542, 0.56613857], dtype=float32)

Gradient Computation

In [47]:
def square(x):
    return tf.multiply(x,x)

with tf.GradientTape(persistent=True) as tape:
    print(square(6.).numpy())

36.0


Bonjour Keras

Defining a sequential model

In [1]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(13,input_dim=7,activation='relu'))
model.add(Dense(7,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

Defining a functional model

In [7]:
from keras.models import Model
from keras.layers import Input, Dense
import numpy as np

# Convert lists to NumPy arrays
data_test = np.array([[1, 2, 3, 4, 5, 6, 7], [2, 3, 4, 5, 6, 7, 8], [3, 4, 5, 6, 7, 8, 9]])
labels_test = np.array([0, 1, 0])
data = np.array([
    [1, 2, 3, 4, 5, 6, 7],
    [2, 3, 4, 5, 6, 7, 8],
    [3, 4, 5, 6, 7, 8, 9],
    [4, 5, 6, 7, 8, 9, 10],
])
labels = np.array([0, 1, 0, 1])

input = Input(shape=(7,))
layer1 = Dense(10,activation='relu')(input)
layer2 = Dense(1,activation='relu')(layer1)
output = Dense(1, activation='sigmoid')(layer2)
#create the model
model = Model(inputs=input, outputs=output)

#compiling the model
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

#training the model
model.fit(x=data, y=labels, epochs=100, batch_size=10)

Epoch 1/100


1/1 [==============================] - 1s 1s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 15ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 0s 0s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 7/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 8/100
1/1 [==============================] - 0s 17ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 10/100
1/1 [==============================] - 0s 4ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 11/100
1/1 [========

In [9]:
#evaluate the model
model.evaluate(x=data_test,y=labels_test)
model.evaluate(x=data, y=labels)


1/1 [==============================] - 0s 46ms/step - loss: 0.6931 - accuracy: 0.5000


[0.6931471824645996, 0.5]

MNIST digit classification using TensorFlow 2.0

In [11]:
import tensorflow as tf

mnist= tf.keras.datasets.mnist
(x_train,y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = tf.cast(x_train/255.0, tf.float32), tf.cast(x_test/255.0,
tf.float32)
y_train, y_test = tf.cast(y_train,tf.int64),tf.cast(y_test,tf.int64)
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=
['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=10)
model.evaluate(x_test, y_test)




11490434/11490434 [==============================] - 61s 5us/step
Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.5919 - accuracy: 0.8449
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2876 - accuracy: 0.9176
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2349 - accuracy: 0.9328
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1999 - accuracy: 0.9423
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1740 - accuracy: 0.9499
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1540 - accuracy: 0.9561
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1381 - accuracy: 0.9605
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1245 - accuracy: 0.9643
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1135 - accuracy: 0.9680
Epoch 10/10
313/

[0.11064107716083527, 0.96670001745224]